Using word2vec to create an unsupervised neural network.

Word2vec is a shallow neural network model for converting words to vectors using distributed representation, each word is represented by many neurons, and each neuron is involved in representing many words. 

Useful for parsing requests written by people, but works well only for larger datasets (i.e. corpus that is several billion words long)

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
def text_cleaner(text):
    #text = re.sub("[\[].*?[\]]", "", text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.split()
    lmz = WordNetLemmatizer()
    text = [lmz.lemmatize(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

In [3]:
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-'+novel+'.txt')
    austen += work

In [4]:
austen = re.sub(r'Chapter \d+','',austen)
austen = re.sub(r'--',' ',austen)

In [5]:
# parse the data into sentences
austen_sent = nltk.sent_tokenize(austen)
#austen_sent = nltk.word_tokenize(austen)

In [6]:
corpus = []
for sent in list(austen_sent):
    cleaned_sentence = text_cleaner(sent)
    cleaned_sentence = nltk.word_tokenize(cleaned_sentence)
    corpus.append(cleaned_sentence)

In [7]:
print(corpus[20])
print('We have {} sentences and {} tokens.'.format(len(corpus), len(austen_sent)))

['one', 'daughter', 'eldest', 'would', 'really', 'given', 'thing', 'much', 'tempted']
We have 17565 sentences and 17565 tokens.


In [12]:
corpus

[['persuasion',
  'jane',
  'austen',
  'sir',
  'walter',
  'elliot',
  'kellynch',
  'hall',
  'somersetshire',
  'man',
  'amusement',
  'never',
  'took',
  'book',
  'baronetage',
  'found',
  'occupation',
  'idle',
  'hour',
  'consolation',
  'distressed',
  'one',
  'faculty',
  'roused',
  'admiration',
  'respect',
  'contemplating',
  'limited',
  'remnant',
  'earliest',
  'patent',
  'unwelcome',
  'sensation',
  'arising',
  'domestic',
  'affair',
  'changed',
  'naturally',
  'pity',
  'contempt',
  'turned',
  'almost',
  'endless',
  'creation',
  'last',
  'century',
  'every',
  'leaf',
  'powerless',
  'could',
  'read',
  'history',
  'interest',
  'never',
  'failed'],
 ['page',
  'favourite',
  'volume',
  'always',
  'opened',
  'elliot',
  'kellynch',
  'hall'],
 ['walter',
  'elliot',
  'born',
  'march',
  'married',
  'july',
  'elizabeth',
  'daughter',
  'james',
  'stevenson',
  'esq'],
 ['south',
  'park',
  'county',
  'gloucester',
  'lady',
  'died'

In [8]:
"loud" in corpus

False

In [9]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    corpus,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

In [10]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))
# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print("\n",model.wv.similarity('loud', 'aloud'))
print("\n",model.wv.similarity('mr', 'miss'))

# One of these things is not like the other...
print("\n",model.doesnt_match("breakfast marriage dinner lunch".split()))

[('shew', 0.5165904760360718), ('miss', 0.47268956899642944), ('attention', 0.4605565071105957), ('hall', 0.4563578963279724), ('received', 0.4457007646560669), ('pressing', 0.44117599725723267), ('looking', 0.43611758947372437), ('handsome', 0.4197090268135071), ('absolute', 0.41104984283447266), ('continuance', 0.40783587098121643)]

 0.67639

 0.6258998

 marriage


/home/glenn/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/home/glenn/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()


In [11]:
vocab

dict_keys(['persuasion', 'jane', 'sir', 'walter', 'elliot', 'kellynch', 'hall', 'somersetshire', 'man', 'amusement', 'never', 'took', 'book', 'found', 'occupation', 'idle', 'hour', 'consolation', 'distressed', 'one', 'roused', 'admiration', 'respect', 'unwelcome', 'sensation', 'domestic', 'affair', 'changed', 'naturally', 'pity', 'contempt', 'turned', 'almost', 'last', 'every', 'could', 'read', 'history', 'interest', 'failed', 'favourite', 'volume', 'always', 'opened', 'born', 'married', 'elizabeth', 'daughter', 'james', 'south', 'park', 'county', 'lady', 'died', 'june', 'anne', 'still', 'son', 'november', 'mary', 'precisely', 'stood', 'hand', 'improved', 'information', 'family', 'word', 'birth', 'charles', 'heir', 'musgrove', 'uppercross', 'day', 'month', 'lost', 'wife', 'followed', 'rise', 'respectable', 'usual', 'term', 'first', 'settled', 'mentioned', 'office', 'high', 'three', 'exertion', 'dignity', 'baronet', 'year', 'forming', 'altogether', 'two', 'handsome', 'arm', 'principal',